In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scanpy as sc

rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False

import NaiveDE
import SpatialDE

Populating the interactive namespace from numpy and matplotlib


In [ ]:
for i in range(10):
    adata = sc.read_h5ad(f"../../data/simulation/svgs/adata_rep_{i}.h5ad")
    adata.obs.set_index('obs_ids', inplace=True)
    adata.obs['x'] = adata.obsm['spatial'][:, 0]
    adata.obs['y'] = adata.obsm['spatial'][:, 1]
    adata.obs['total_counts'] = adata.layers['counts'].sum(axis=1)
    adata.obs.head()
    counts = adata.layers['counts'].todense().astype(np.float32)
    counts = pd.DataFrame(data=counts, index=adata.obs_names, columns=adata.var_names)
    norm_expr = NaiveDE.stabilize(counts.T).T
    resid_expr = NaiveDE.regress_out(adata.obs, norm_expr.T, 'np.log(total_counts)').T
    X = adata.obs[['x', 'y']]
    df = SpatialDE.run(X, resid_expr)
    df.set_index("g", inplace=True)
    df = df.loc[adata.var_names]
    df['spatially_variable'] = adata.var.spatially_variable.astype(np.int).values
    df = df[['qval', 'spatially_variable']]
    df.to_csv(f"./svgs/rep_{i}.csv")